In [ ]:

# 졸업자 수 및 진학률 통합 전처리 (2010~2023)
import pandas as pd
from difflib import get_close_matches

# 1. 파일 경로 및 연도 정의
file_paths = {...}  # 연도: 파일경로 (이후 실제 코드에서 사용한 경로 매핑 필요)
rate_cols = {2010: 18, 2011: 18, 2012: 18}  # 연도별 진학률 열 인덱스 (기타는 기본값 22)
school_col = 1  # 학교명 열
male_col, female_col = 2, 3

# 2. 전처리 함수 정의
def preprocess(file, year, rate_col):
    df = pd.read_excel(file, sheet_name="Sheet1", header=None)
    df = df.iloc[7:, [school_col, male_col, female_col, rate_col]].copy()
    df.columns = ['학교명', '졸업자_남', '졸업자_여', '진학률']
    df['졸업자 수'] = pd.to_numeric(df['졸업자_남'], errors='coerce') + pd.to_numeric(df['졸업자_여'], errors='coerce')
    df['진학률'] = pd.to_numeric(df['진학률'], errors='coerce')
    df['기준년도'] = year
    df = df[['기준년도', '학교명', '졸업자 수', '진학률']]
    df = df[df['학교명'].notna()]
    df = df[~df['학교명'].astype(str).str.contains("사이버")]
    df = df[~((df['졸업자 수'] == 0) & (df['진학률'] == 0))]
    return df

# 3. 연도별 데이터 전처리
preprocessed = {}
for year, path in file_paths.items():
    rate_col = rate_cols.get(year, 22)
    preprocessed[year] = preprocess(path, year, rate_col)

# 4. 유사 매칭 기반 정규화
standard_names = list(preprocessed[2023]['학교명'].unique())
mapping = {}
for df in preprocessed.values():
    for name in df['학교명'].unique():
        match = get_close_matches(name, standard_names, n=1, cutoff=0.9)
        if match:
            mapping[name] = match[0]

# 5. 정규화 및 통합
normalized = []
for year, df in preprocessed.items():
    df_copy = df.copy()
    df_copy['학교명'] = df_copy['학교명'].map(mapping)
    df_copy = df_copy[df_copy['학교명'].notna()]
    normalized.append(df_copy)

merged = pd.concat(normalized, ignore_index=True)
merged.to_csv("졸업자수_진학률_2010_2023_유사통합_utf8sig.csv", index=False, encoding='utf-8-sig')
merged.head()
